# Basic Vector Search from Scratch

For this exercise we will implement basic vector search
from scratch with just numpy.<br/>
This will give us a feel
for what's happening under the hood in vector databases.

In [1]:
# !pip install numpy pytest [This is preinstalled for you in the workspace]

## Euclidean distance

There are many ways to measure the distance between two vectors.
Let's write a function that computes the `Euclidean distance` 
between vectors. 

This function should take as input two vectors and return
the euclidean distance between them.

For more details you can read this [kaggle page](https://www.kaggle.com/code/paulrohan2020/euclidean-distance-and-normalizing-a-vector)


In [1]:
import numpy as np

ModuleNotFoundError: No module named 'numpy'

In [30]:
def euclidean_distance(v1: np.ndarray, v2: np.ndarray) -> float:
    """
    Compute the Euclidean distance between two vectors.

    Parameters
    ----------
    v1 : np.ndarray
        First vector.
    v2 : np.ndarray
        Second vector.

    Returns
    -------
    float
        Euclidean distance between `v1` and `v2`.
    """
    distance = np.linalg.norm(v1 - v2)
    
    return distance

## KNN search

Using the distance function you just wrote, write a function that 
finds the k-nearest neighbors of a query vector.

This function should take as input a query vector, a 2d array of database vectors,
and an integer k the number of nearest neighbors to return. And it should return 
the vectors that are the k-nearest neighbors of the query vector.


In [31]:
def find_nearest_neighbors(query: np.ndarray,
                           vectors: np.ndarray,
                           k: int = 1) -> np.ndarray:
    """
    Find k-nearest neighbors of a query vector.

    Parameters
    ----------
    query : np.ndarray
        Query vector.
    vectors : np.ndarray
        Vectors to search.
    k : int, optional
        Number of nearest neighbors to return, by default 1.

    Returns
    -------
    np.ndarray
        The `k` nearest neighbors of `query` in `vectors`.
    """
    distances = []
    
    # Compute the distance between the query vector and each vector in the database
    for i, db_vector in enumerate(vectors):
        distance = euclidean_distance(query_, db_vector)
        distances.append((distance, i))
    
    # Sort distances
    distances.sort(key=lambda x: x[0])
    
    # Select the k smallest distances
    nearest_neighbors = distances[:k]
    
    # Extract the distances and indices
    nearest_indices = [dist[1] for dist in nearest_neighbors]
    
    return vectors[nearest_indices]
        

In [32]:
a=np.array([[1,2],[3,4],[5,6],[7,8]])
a

array([[1, 2],
       [3, 4],
       [5, 6],
       [7, 8]])

In [33]:
a[[1,3]]

array([[3, 4],
       [7, 8]])

## Other distance metrics

For this problem we'll write a new distance function and modify 
our nearest neighbors function to accept a distance metric.


Write a function that computes the [cosine distance](ttps://en.wikipedia.org/wiki/Cosine_similarity) between vectors.

In [41]:
from typing import Union

def cosine_distance(v1: np.ndarray, v2: np.ndarray) -> Union[float, np.ndarray]:
    """
    Compute the cosine distance between two vectors.

    Parameters
    ----------
    v1 : np.ndarray
        First vector.
    v2 : np.ndarray
        Second vector.

    Returns
    -------
    float
        Cosine distance between `v1` and `v2`.
    """
    cos_sim = np.dot(v1,v2) / (np.linalg.norm(v1) * np.linalg.norm(v2))    
    cos_dist = 1 - cos_sim
    return cos_dist

**HINT** Please make sure you understand the difference between cosine similarity and cosine distance

Now, rewrite the `find_nearest_neighbors` function to accept a distance metric so you can use either Euclidean or Cosine distance

In [46]:
def find_nearest_neighbors(query: np.ndarray,
                           vectors: np.ndarray,
                           k: int = 1,
                           distance_metric="euclidean") -> np.ndarray:
    """
    Find k-nearest neighbors of a query vector with a configurable
    distance metric.

    Parameters
    ----------
    query : np.ndarray
        Query vector.
    vectors : np.ndarray
        Vectors to search.
    k : int, optional
        Number of nearest neighbors to return, by default 1.
    distance_metric : str, optional
        Distance metric to use, by default "euclidean".

    Returns
    -------
    np.ndarray
        The `k` nearest neighbors of `query` in `vectors`.
    """
    distances = []
    
    for i, db_vector in enumerate(vectors):
        if distance_metric=="cosine":
            distances.append((cosine_distance(query, db_vector),i))
                             
        elif distance_metric=="euclidean":
            distances.append((euclidean_distance(query, db_vector),i))
                             
        else:
            raise exception("Invalid distance metric")
        
    distances.sort(key = lambda x: x[0])
    print(distances)            
    distances=distances[:k]
    print(distances)               
    indices = [dist[1] for dist in distances]
                             
    return vectors[indices]
                             
    
            
    

## Exploration

Now that we have a nearest neighbors function that accepts a distance metric, <br/>
let's explore the differences between Euclidean distance and cosine distance.

Would you expect same or different answers?

In [47]:
# You might find this function useful

def generate_vectors(num_vectors: int, num_dim: int,
                     normalize: bool = True) -> np.ndarray:
    """
    Generate random embedding vectors.

    Parameters
    ----------
    num_vectors : int
        Number of vectors to generate.
    num_dim : int
        Dimensionality of the vectors.
    normalize : bool, optional
        Whether to normalize the vectors, by default True.

    Returns
    -------
    np.ndarray
        Randomly generated `num_vectors` vectors with `num_dim` dimensions.
    """
    vectors = np.random.rand(num_vectors, num_dim)
    if normalize:
        vectors /= np.linalg.norm(vectors, axis=1, keepdims=True)
    return vectors

In [48]:
vecs = generate_vectors(5, 2)
vecs

array([[0.60418738, 0.79684227],
       [0.2556094 , 0.96678014],
       [0.93672826, 0.35005738],
       [0.98107844, 0.19361066],
       [0.94508009, 0.32683884]])

In [54]:
find_nearest_neighbors(query=[0.98,0.20],
                           vectors=vecs,
                           k =1,
                           distance_metric="cosine")

[(2.0969130698378358e-05, 3), (0.008651995175799843, 4), (0.012192372799171736, 2), (0.24867815836057894, 0), (0.5562355052695225, 1)]
[(2.0969130698378358e-05, 3)]


array([[0.98107844, 0.19361066]])